# Script for a period

In [34]:
import requests
import json
import pandas as pd
from datetime import date
import openpyxl
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time

load_dotenv()

True

In [35]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv("KeySmart")

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [36]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)
response_smart = requests.get(url, headers=headers_smart)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    data_smart = response_smart.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code,response_smart.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text,response_smart.text }")

Data retrieved successfully


In [37]:
# Flatten the JSON data
df_guten = pd.json_normalize(
    data_guten['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_giper = pd.json_normalize(
    data_giper['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_kitchen = pd.json_normalize(
    data_kitchen['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_smart = pd.json_normalize(
    data_smart['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)

# Convert 'changeTime' to datetime format
df_guten['changeTime'] = pd.to_datetime(df_guten['changeTime'])
df_giper['changeTime'] = pd.to_datetime(df_giper['changeTime'])
df_kitchen['changeTime'] = pd.to_datetime(df_kitchen['changeTime'])
df_smart['changeTime'] = pd.to_datetime(df_smart['changeTime'])

# Filter rows where the year of 'changeTime' is 2024 or later
df_guten = df_guten[df_guten['changeTime'].dt.year >= 2024]
df_giper = df_giper[df_giper['changeTime'].dt.year >= 2024]
df_kitchen = df_kitchen[df_kitchen['changeTime'].dt.year >= 2024]
df_smart = df_smart[df_smart['changeTime'].dt.year >= 2024]

# Reset the index and drop the old index
df_guten = df_guten.reset_index(drop=True)
df_giper = df_giper.reset_index(drop=True)
df_kitchen = df_kitchen.reset_index(drop=True)
df_smart = df_smart.reset_index(drop=True)

# Display the updated DataFrame
#df_guten
#df_giper
#df_kitchen

In [38]:
#unique_change_times = df['changeTime'].unique()
#print(unique_change_times)

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [39]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks_guten = [df_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_guten), chunk_size)]
campaign_chunks_giper = [df_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_giper), chunk_size)]
campaign_chunks_kitchen = [df_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_kitchen), chunk_size)]
campaign_chunks_smart = [df_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_smart), chunk_size)]

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# List to store all campaign data
all_campaign_data_guten = []
all_campaign_data_giper = []
all_campaign_data_kitchen = []
all_campaign_data_smart = []

# Iterate over each chunk and send a POST request
for idx, (chunk_guten, chunk_giper, chunk_kitchen,chunk_smart) in enumerate(zip(campaign_chunks_guten, campaign_chunks_giper, campaign_chunks_kitchen,campaign_chunks_smart)):
    
    # Send the POST request
    response_guten = requests.post(url, params=query_params, json=chunk_guten, headers=headers_guten)
    time.sleep(1)
    response_giper = requests.post(url, params=query_params, json=chunk_giper, headers=headers_giper)
    time.sleep(1)
    response_kitchen = requests.post(url, params=query_params, json=chunk_kitchen, headers=headers_kitchen)
    time.sleep(1)
    response_smart = requests.post(url, params=query_params, json=chunk_smart, headers=headers_smart)
    # Add a delay to avoid hitting API rate limits
    time.sleep(1)
    
    # Check the response status
    if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
        # Parse the JSON response
        data_guten = response_guten.json()
        data_giper = response_giper.json()
        data_kitchen = response_kitchen.json()
        data_smart = response_smart.json()
        all_campaign_data_guten.extend(data_guten)
        all_campaign_data_giper.extend(data_giper)
        all_campaign_data_kitchen.extend(data_kitchen)
        all_campaign_data_smart.extend(data_smart)
        print("Data retrieved successfully")
        #print(f"Response for Chunk {idx + 1}: {data_guten}")
    else:
        print(f"Error for Chunk {idx + 1}: {response_guten.status_code,response_giper.status_code,response_kitchen.status_code,response_smart.status_code}, {response_guten.text,response_giper.text,response_kitchen.text,response_smart.text}")
        
        
campaigns_guten = pd.DataFrame(all_campaign_data_guten)
campaigns_giper = pd.DataFrame(all_campaign_data_giper)
campaigns_kitchen = pd.DataFrame(all_campaign_data_kitchen)
campaigns_smart = pd.DataFrame(all_campaign_data_smart)

campaigns_guten['Project'] = 'WB-GutenTech'
campaigns_giper['Project'] = 'WB-ГиперМаркет'
campaigns_kitchen['Project'] = 'WB-KitchenAid'
campaigns_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen,campaigns_smart], ignore_index=True)

combined_campaigns['Marketplace'] = 'Wildberries'

print("Combined Campaign Data")
#campaigns_kitchen

Data retrieved successfully
Combined Campaign Data


In [40]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2024-12-23,2025-01-07,Metz_9 SKU_23.12.2024,22255058,Кампания на паузе,Аукцион,WB-GutenTech,Wildberries
1,2100-01-01,2024-12-18,2024-12-18,Пылесос Zelmer ZVC9130M от 18.12.2024,22165389,Кампания на паузе,Аукцион,WB-GutenTech,Wildberries
2,2100-01-01,2024-11-26,2025-01-19,AND_Массажер MH-103_26.11.2024_ВЧ,21639902,Кампания на паузе,Аукцион,WB-GutenTech,Wildberries
3,2024-12-09,2024-10-04,2024-11-15,ELIKOR_Врезные блоки_2 SKU_04.10.2024,20397195,Кампания завершена,Автоматическая кампания,WB-GutenTech,Wildberries
4,2024-12-09,2024-10-03,2024-11-07,DEERMA_Увлажнители_03.10.2024,20354793,Кампания завершена,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
101,2024-06-23,2024-03-21,2024-04-18,Насадки Oral-B Автореклама,15562382,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries
102,2100-01-01,2024-03-21,2025-02-20,Электробритвы Braun_22.10.2024,15562363,Идут показы,Автоматическая кампания,WB-Smart-Market,Wildberries
103,2024-10-11,2024-03-21,2024-07-02,Электрические зубные щетки Oral-B Поиск + Каталог,15562321,Кампания завершена,Аукцион,WB-Smart-Market,Wildberries
104,2024-10-15,2024-03-21,2024-08-27,Эпиляторы Braun_7 SKU_27.08.2024,15562282,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [41]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Define the date interval
date_interval = {
    "begin": "2025-03-01",  # Replace with your start date
    "end": "2025-03-05"     # Replace with your end date
}

# Create chunks of 100 campaign IDs each
chunk_size = 100

### Guten

In [42]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_guten), chunk_size)]

# List to store all campaign data
all_campaign_data_guten = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_guten, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_guten.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_guten_df = pd.json_normalize(all_campaign_data_guten)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_guten, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

TypeError: 'NoneType' object is not iterable

### Giper

In [ ]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_giper), chunk_size)]

# List to store all campaign data
all_campaign_data_giper = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_giper, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_giper.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_giper_df = pd.json_normalize(all_campaign_data_giper)

# Save the combined data to a text file
#with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    #json.dump(all_campaign_data_giper, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
#print("Combined Campaign Data")

Data retrived successfully


### Kitchen

In [ ]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_kitchen), chunk_size)]

# List to store all campaign data
all_campaign_data_kitchen = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_kitchen, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_kitchen.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_kitchen_df = pd.json_normalize(all_campaign_data_kitchen)

Data retrived successfully


### Smart

In [ ]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_smart), chunk_size)]

# List to store all campaign data
all_campaign_data_smart = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_smart, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_smart.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_smart_df = pd.json_normalize(all_campaign_data_smart)

Data retrived successfully


In [ ]:
campaign_smart_df

,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,boosterStats,advertId,interval.begin,interval.end
0,7,0,0.00,0.00,0.98,0,0,0.00,0,0,"[{'date': '2025-02-15T03:00:00+03:00', 'views'...","[{'date': '2025-02-15T00:00:00Z', 'nm': 288465...",21693448,2025-02-01,2025-02-28
1,107255,6351,5.92,1.81,11493.44,990,163,2.57,163,1084678,"[{'date': '2025-02-01T03:00:00+03:00', 'views'...","[{'date': '2025-02-05T00:00:00Z', 'nm': 235490...",20822563,2025-02-01,2025-02-28
2,32280,941,2.92,4.72,4440.61,41,4,0.43,4,4424,"[{'date': '2025-02-12T03:00:00+03:00', 'views'...","[{'date': '2025-02-12T00:00:00Z', 'nm': 288465...",23236382,2025-02-12,2025-02-28
3,95530,4828,5.05,2.15,10358.35,611,141,2.92,141,673698,"[{'date': '2025-02-01T03:00:00+03:00', 'views'...","[{'date': '2025-02-05T00:00:00Z', 'nm': 235350...",19597465,2025-02-01,2025-02-28
4,64862,4976,7.67,1.47,7306.90,695,84,1.69,86,1242267,"[{'date': '2025-02-01T03:00:00+03:00', 'views'...","[{'date': '2025-02-01T00:00:00Z', 'nm': 206386...",20822524,2025-02-01,2025-02-28
5,125294,4654,3.71,3.75,17441.61,235,39,0.84,46,51913,"[{'date': '2025-02-12T03:00:00+03:00', 'views'...","[{'date': '2025-02-12T00:00:00Z', 'nm': 288465...",23236144,2025-02-12,2025-02-28
6,158,10,6.33,2.26,22.59,1,0,0.00,0,0,"[{'date': '2025-02-13T03:00:00+03:00', 'views'...","[{'date': '2025-02-13T00:00:00Z', 'nm': 288465...",21693077,2025-02-01,2025-02-28
7,52243,842,1.61,8.46,7125.73,72,23,2.73,24,83788,"[{'date': '2025-02-12T03:00:00+03:00', 'views'...","[{'date': '2025-02-12T00:00:00Z', 'nm': 313543...",23236221,2025-02-12,2025-02-28
8,101090,5581,5.52,1.91,10687.25,829,203,3.64,205,1977989,"[{'date': '2025-02-01T03:00:00+03:00', 'views'...","[{'date': '2025-02-05T00:00:00Z', 'nm': 205352...",15562363,2025-02-01,2025-02-28


In [ ]:
# Flatten the JSON data
flattened_data_guten = []
flattened_data_giper = []
flattened_data_kitchen = []
flattened_data_smart = []


for entry in all_campaign_data_guten:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_guten.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_giper:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_giper.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

for entry in all_campaign_data_kitchen:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_kitchen.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_smart:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_smart.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                

# Create a DataFrame
df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)
# Convert the 'date' column to datetime
df_guten["date"] = pd.to_datetime(df_guten["date"])
df_giper["date"] = pd.to_datetime(df_giper["date"])
df_kitchen["date"] = pd.to_datetime(df_kitchen["date"])
df_smart["date"] = pd.to_datetime(df_smart["date"])
# Remove timezone information (make it timezone-naive)
df_guten["date"] = df_guten["date"].dt.tz_localize(None)
df_giper["date"] = df_giper["date"].dt.tz_localize(None)
df_kitchen["date"] = df_kitchen["date"].dt.tz_localize(None)
df_smart["date"] = df_smart["date"].dt.tz_localize(None)

# Display the DataFrame
df_guten

,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-02-01 03:00:00,235887151,Массажер для тела электрический AND MH-103 с и...,2,0,0.00,0.00,0.18,0,0,0.0,0,0,21624850
1,2025-02-02 03:00:00,235887151,Массажер для тела электрический AND MH-103 с и...,1,0,0.00,0.00,0.09,0,0,0.0,0,0,21624850
2,2025-02-03 03:00:00,235887151,Массажер для тела электрический AND MH-103 с и...,1,0,0.00,0.00,0.08,0,0,0.0,0,0,21624850
3,2025-02-03 03:00:00,235887151,Массажер для тела электрический AND MH-103 с и...,1,0,0.00,0.00,0.09,0,0,0.0,0,0,21624850
4,2025-02-04 03:00:00,235887151,Массажер для тела электрический AND MH-103 с и...,1,1,100.00,0.09,0.09,0,0,0.0,0,0,21624850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3130,2025-02-28 03:00:00,303419149,Электробритва мужская для лица Supercar аккуму...,96,1,1.04,25.48,25.48,0,0,0.0,0,0,22255122
3131,2025-02-28 03:00:00,303419150,Электробритва мужская для лица Firm Stone акку...,18,0,0.00,0.00,4.67,0,0,0.0,0,0,22255122
3132,2025-02-28 03:00:00,303419144,Электробритва мужская для лица Traveller аккум...,35,0,0.00,0.00,8.99,0,0,0.0,0,0,22255122
3133,2025-02-28 03:00:00,303419147,Электробритва мужская для лица Camerist аккуму...,21,0,0.00,0.00,5.50,0,0,0.0,0,0,22255122


In [ ]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_guten = (
    df_guten.groupby([df_guten["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_giper = (
    df_giper.groupby([df_giper["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_kitchen = (
    df_kitchen.groupby([df_kitchen["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_smart = (
    df_smart.groupby([df_smart["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)


# Rename the 'date' column to 'day' for clarity
df_grouped_guten.rename(columns={"date": "day"}, inplace=True)
df_grouped_giper.rename(columns={"date": "day"}, inplace=True)
df_grouped_kitchen.rename(columns={"date": "day"}, inplace=True)
df_grouped_smart.rename(columns={"date": "day"}, inplace=True)

df_grouped_guten['Project'] = 'WB-GutenTech'
df_grouped_giper['Project'] = 'WB-ГиперМаркет'
df_grouped_kitchen['Project'] = 'WB-KitchenAid'
df_grouped_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen, df_grouped_smart], ignore_index=True)
df_grouped_combined_campaigns['Marketplace'] = 'Wildberries'

# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,25962484,2025-02-01 03:00:00,Кухонные весы Kitchen Energy,418,4,1.313333,14.683333,66.23,0,0,0.0,0,0,18870701,WB-GutenTech,Wildberries
1,43371059,2025-02-01 03:00:00,Кухонные весы Kitchen EcoMate,564,14,1.586667,4.900000,89.18,0,0,0.0,0,0,18870701,WB-GutenTech,Wildberries
2,121061623,2025-02-01 03:00:00,Электронные кухонные весы LX 20,45,1,1.283333,0.813333,4.31,0,0,0.0,0,0,18870701,WB-GutenTech,Wildberries
3,125425997,2025-02-01 03:00:00,Кофемашина автоматическая зерновая BCC01RDMEU,1,0,0.000000,0.000000,0.06,0,0,0.0,0,0,21041312,WB-GutenTech,Wildberries
4,128210218,2025-02-01 03:00:00,Миксер кухонный ручной HMF01CREU,1169,29,1.490000,7.303333,222.41,1,0,0.0,0,0,21041248,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7331,288465484,2025-02-28 03:00:00,Набор Mach3 с лосьоном после бритья,1,0,0.000000,0.000000,0.05,0,0,0.0,0,0,23236144,WB-Smart-Market,Wildberries
7332,288465485,2025-02-28 03:00:00,Набор Fusion5 ProGlide Flexball с гелем для бр...,327,6,2.736667,13.330000,46.29,0,0,0.0,0,0,23236144,WB-Smart-Market,Wildberries
7333,288465487,2025-02-28 03:00:00,Набор Series Пена для бритья 200мл + Мочалка,1123,11,0.956667,5.600000,106.04,1,0,0.0,0,0,23236144,WB-Smart-Market,Wildberries
7334,313543155,2025-02-28 03:00:00,,1061,11,0.703333,11.990000,147.12,1,0,0.0,0,0,23236221,WB-Smart-Market,Wildberries


In [ ]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25962484,2025-02-01 03:00:00,Кухонные весы Kitchen Energy,418,4,66.23,0,0,0,0,18870701,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Весы кух_2 SKU_25.07.24,Кампания на паузе,Автоматическая кампания
1,43371059,2025-02-01 03:00:00,Кухонные весы Kitchen EcoMate,564,14,89.18,0,0,0,0,18870701,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Весы кух_2 SKU_25.07.24,Кампания на паузе,Автоматическая кампания
2,121061623,2025-02-01 03:00:00,Электронные кухонные весы LX 20,45,1,4.31,0,0,0,0,18870701,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Весы кух_2 SKU_25.07.24,Кампания на паузе,Автоматическая кампания
3,125425997,2025-02-01 03:00:00,Кофемашина автоматическая зерновая BCC01RDMEU,1,0,0.06,0,0,0,0,21041312,WB-GutenTech,Wildberries,2100-01-01,2024-10-31,2024-11-27,SMEG_Кофемашина_31.10.2024,Кампания на паузе,Автоматическая кампания
4,128210218,2025-02-01 03:00:00,Миксер кухонный ручной HMF01CREU,1169,29,222.41,1,0,0,0,21041248,WB-GutenTech,Wildberries,2100-01-01,2024-10-31,2024-12-28,SMEG_Миксер ручной_31.10.2024,Кампания на паузе,Автоматическая кампания
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7331,288465484,2025-02-28 03:00:00,Набор Mach3 с лосьоном после бритья,1,0,0.05,0,0,0,0,23236144,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-24,Gillette_Бритвы + наборы_12.02.2025,Кампания на паузе,Автоматическая кампания
7332,288465485,2025-02-28 03:00:00,Набор Fusion5 ProGlide Flexball с гелем для бр...,327,6,46.29,0,0,0,0,23236144,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-24,Gillette_Бритвы + наборы_12.02.2025,Кампания на паузе,Автоматическая кампания
7333,288465487,2025-02-28 03:00:00,Набор Series Пена для бритья 200мл + Мочалка,1123,11,106.04,1,0,0,0,23236144,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-24,Gillette_Бритвы + наборы_12.02.2025,Кампания на паузе,Автоматическая кампания
7334,313543155,2025-02-28 03:00:00,,1061,11,147.12,1,0,0,0,23236221,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-21,Gillette_Кассеты_12.02.2025,Идут показы,Автоматическая кампания


## Insert the data

In [ ]:
password = os.getenv('ClickHouse')

In [ ]:
# Define connection parameters
client = get_client(
    host='rc1a-vk5i3icccvmfk6cm.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                           # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [ ]:
# Define the table name
table_name = 'campaign_data_wb'

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in df_final.to_numpy()]

# Define the column names
column_names = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks', 'sum_price',
    'advertId', 'Project', 'Marketplace', 'endTime', 'createTime', "startTime", 'name_campaign', 'status', 'type'
]

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=column_names)

print("Data inserted successfully!")

Data inserted successfully!
